In [41]:
import numpy as np
import numpy.random as npr
import pandas as pd

In [12]:
# Importantions (temporarily here during tests)
from scipy.ndimage.measurements import label
from scipy.spatial.distance import cdist
from scipy import ndimage

array = np.zeros([3 * 3], dtype=np.int)
array[[0]] = 1
array[[8]] = 1
array = np.reshape(array, (3, 3))
structure = np.ones((3, 3), dtype=np.int)
labeled, ncomponents = label(array, structure)
print(labeled, "\n",ncomponents)

[[1 0 0]
 [0 0 0]
 [0 0 2]] 
 2


In [35]:
class ToyForest(object):
    def __init__(self, rows, cols, vol, demand):
        # Dimensions
        self._Rows = rows
        self._Cols = cols
        self._NCells = rows * cols
        
        # Time
        self._Year = 1
        
        # Boolean flags
        self._demandC = True
        self._evenFlowC = True
        self._adjC = True
        self._finalC = True
            
        # Volume
        self._VolCells = np.full(self._NCells, vol)
        
        # FPV 
        self._FPVMatrix = np.random.randint(100, size=self._NCells)
        
        # Demand
        self._PeriodDemand = demand
        
    @property
    def getYear(self):
        return self._Year
    
    @property 
    def getDemand(self):
        return self._PeriodDemand
        
    @property
    def getVol(self):
        return self._VolCells
    
    @property 
    def getFPVMatrix(self):
        return self._FPVMatrix
        
    @property
    def shape(self):
        return (self._Rows, self._Cols)

In [39]:
# Forest object
demand = [2,2,2]
Forest = ToyForest(3,3,1,demand)

# FPV Matrix 
print("FPV Matrix:\n", np.reshape(Forest.getFPVMatrix, (Forest.shape)))
      
# Volume Matrix
print("Vol Matrix:\n", np.reshape(Forest.getVol, (Forest.shape)))
      
# Demand
print("Demand:\n", Forest.getDemand)
      

FPV Matrix:
 [[47 76 49]
 [79 53  3]
 [81 90 87]]
Vol Matrix:
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
Demand:
 [2, 2, 2]


In [40]:
# Genetic code
"""
Binary genetic code (1 = harvested, 0 = non harvested)
"""
def geneCode:
    genes = [0,1]
    return genes


# Individual (Genetic algorithm)
"""
Array/Matrix of ones and zeros representing if a cell is harvested or not
"""
def initIndividual(ncells)
    individual = npr.randint(2, ncells)
    return individual


# Mutation 
"""
Swap 1 and 0, replace 0 by 1, replace 1 by 0 with different chances.
"""
def mutate(individual, alpha=0.15, beta=0.15, gamma=0.15):
    rn = npr.rand(3)
    
    if rn[0] <= alpha: 
        individual = 
    if rn[1] <= beta: 
        individual =
    if rn[2] <= gamma: 
        individual = 

In [2]:
# Adjacency fn V1
def AdjacencyFnV1(self, actionArray):
    # Initialize penalty = 0 (total distance from largest patch)
    adjDistance = 0
    #print("------Debugging Adjacency FnV1-----")

    # Importantions (temporarily here during tests)
    from scipy.ndimage.measurements import label
    from scipy.spatial.distance import cdist
    from scipy import ndimage

    # Distance between features function (connected components inside the array)
    def feature_dist(input):
        """
        Takes a labeled array as returned by scipy.ndimage.label and 
        returns an intra-feature distance matrix.
        """
        I, J = np.nonzero(input)
        labels = input[I,J]
        coords = np.column_stack((I,J))

        sorter = np.argsort(labels)
        labels = labels[sorter]
        coords = coords[sorter]

        sq_dists = cdist(coords, coords, 'chebyshev')#'cityblock')#'sqeuclidean')

        start_idx = np.flatnonzero(np.r_[1, np.diff(labels)])
        nonzero_vs_feat = np.minimum.reduceat(sq_dists, start_idx, axis=1)
        feat_vs_feat = np.minimum.reduceat(nonzero_vs_feat, start_idx, axis=0)

        return np.sqrt(feat_vs_feat)

    # Idea: Calculate the number of components, find the largest one, compute the distance (chebyshev)
    #       from each component to it, to penalyze the reward function
    array = np.zeros([self._Rows * self._Cols], dtype=np.int)
    array[[actionArray]] = 1
    array = np.reshape(array, (self._Rows, self._Cols))
    structure = np.ones((3, 3), dtype=np.int)
    labeled, ncomponents = label(array, structure)
    #print("Testing Adjacency function V1:\n", labeled)
    #print("N components:", ncomponents)


    # If more than 1 component, make calculations for penalty 
    # (CP: Can modify the penalty to account the volume of the non-connected patches, weighted by distance
    #      from the largest connected component)
    if ncomponents > 1:

        # Calculate the size of each component
        sizeComponents = ndimage.sum(array, labeled, index=[i for i in range(1, ncomponents+1)])
        #print("Size components:", sizeComponents)
        largestComp = np.argmax(sizeComponents)
        #print("Largest component:", largestComp + 1)

        # Compute the total distance from maximum patch to the rest of the components
        adjDistance = np.sum(feature_dist(labeled)[largestComp,])
        #print("Adj Distance:", adjDistance)


    return adjDistance

In [3]:
# Reward function
def RewardFN_Tactical(self, state, action, new_burnt, done=False):
    actionArray = [i-1 for i in action]

    # Economic value of a cell (need to add a harvesting cost to measure the utility, just volume for the moment)
    econF = 0
    if np.max(actionArray) > -1:    # if -1 then no cell is harvested
        econF = np.sum(self._VolCells[actionArray])

    # BurntCells by the end of the period penalty
    burntF = 0
    if len(new_burnt) > 0:
        burntF = np.sum(self._VolCells[[i-1 for i in new_burnt]])

    # Demand constraint: Penalize non-satisfied demand (CP: and excess, for the moment)
    demandF = 0
    if self._demandC:
        if np.sum(self._VolCells[actionArray]) != self._PeriodDemand[self._Year - 2]:
            demandF = np.abs(self._PeriodDemand[self._Year - 2] - np.sum(self._VolCells[actionArray]))


    # Even Flow constraint: if t>0, check previous production level within an alpha %
    # Penalize by the deviation
    evenflowF = 0
    if self._evenFlowC and self._Year > 2:
        if self._verbose:
            print("Year:", self._Year)
            print("self previous action:", self._previous_action)
        prev_actionArray = [i-1 for i in self._previous_action]

        # UB
        if np.sum(self._VolCells[actionArray]) > np.sum(self._VolCells[prev_actionArray]) * (1 + self._alpha):
            evenflowF += np.sum(self._VolCells[actionArray]) - \
                         np.sum(self._VolCells[prev_actionArray]) * (1 + self._alpha)

        # LB
        elif np.sum(self._VolCells[actionArray]) < np.sum(self._VolCells[prev_actionArray]) * (1 - self._alpha):
            evenflowF += np.sum(self._VolCells[prev_actionArray]) * (1 - self._alpha) - \
                         np.sum(self._VolCells[actionArray])

    # Adjacency (CP: check function, penalizing by distance between patches for the moment)
    adjF = 0
    if self._adjC:
        adjF = self.AdjacencyFnV1(actionArray)

    # Done factor: Total available at the end of the episode bonus
    doneF = 0
    if done and self._finalC:
        doneF = np.sum(self._VolCells[[i-1 for i in self._AvailCells_Set]])


    # Total Reward of the period 
    # (volume harvested - burned - non-satisfied demand - flow deviation - adj violation + final bonus)
    reward = econF - burntF - demandF - evenflowF - adjF + doneF

    if self._nooutput == False:
        print("Reward components:", econF, burntF, demandF, evenflowF, adjF, doneF)

    # Next simulation (if we passed the horizon or the episode is done)
    #if self._Year > self._TotalYears or self._done = True:
    #    self._Sim += 1

    return reward